### Neural Network from scratch

In [21]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# dataset load
data = pd.read_csv("datasets/diabetes2.csv")

# data prepare
X = data.drop('Outcome', axis='columns')
y = data['Outcome'] # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
data.describe()


# model create
class NeuralNetworkFromScratch:
    # X -> I -> W1, b1 -> H -> w2, b2 -> O
    def __init__(self, input_layer_size, hidden_layer_size, output_layer_size):
        self.weights1 = np.random.rand(input_layer_size, hidden_layer_size)
        self.bias1 = np.zeros((1, hidden_layer_size))
        self.weights2 = np.random.rand(hidden_layer_size, output_layer_size)
        self.bias2 = np.zeros((1, output_layer_size))
        
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def sigmoid_derivative(self, z):
        return z*(1-z)
        
    
    def forward(self, x):
        self.hidden = self.sigmoid(np.dot(x, self.weights1) + self.bias1)
        self.output = self.sigmoid(np.dot(self.hidden, self.weights2) + self.bias2)
        return self.output
    
    def backward(self, X, y, learning_rate):
        error = y-self.output
        delta_output = error*self.sigmoid_derivative(self.output)
        error_hidden = delta_output.dot(self.weights2.T)
        delta_hidden = error_hidden*self.sigmoid_derivative(self.hidden)
        
        self.weights2 += self.hidden.T.dot(delta_output)*learning_rate
        self.bias2 += np.sum(delta_output, axis=0, keepdims=True)*learning_rate
        
        self.weights1 += X.T.dot(delta_hidden)*learning_rate
        self.bias1 += np.sum(delta_hidden, axis=0, keepdims=True)*learning_rate
    
    def fit(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, learning_rate)
            if epoch%1000==0:
                loss = np.mean(np.square(y-output))
                print(f"Loss: {loss}, epoch: {epoch}")
    
    def predict(self, x):
        return np.round(self.forward(x))

input_layer_size = X_train.shape[1]
hidden_layer_size = 1
output_layer_size = 1

model = NeuralNetworkFromScratch(input_layer_size, hidden_layer_size, output_layer_size)

# model train
model.fit(X_train.to_numpy(), y_train.to_numpy().reshape(-1,1),learning_rate=0.001, epochs=10000)

# model predict
y_pred = model.predict(X_test)

# evaluate
acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)
matrix = confusion_matrix(y_test, y_pred)

print("Accuracy: ", acc)
print("Classification report: \n", report)
print("Confusion matrix: \n", matrix)


Loss: 0.28524979156992597, epoch: 0
Loss: 0.22973331404321554, epoch: 1000
Loss: 0.22973331404321554, epoch: 2000
Loss: 0.22973331404321554, epoch: 3000
Loss: 0.22973331404321554, epoch: 4000
Loss: 0.22973331404321554, epoch: 5000
Loss: 0.22973331404321554, epoch: 6000
Loss: 0.22973331404321554, epoch: 7000
Loss: 0.22973331404321554, epoch: 8000
Loss: 0.22973331404321554, epoch: 9000
Accuracy:  0.6770833333333334
Classification report: 
               precision    recall  f1-score   support

           0       0.68      1.00      0.81       130
           1       0.00      0.00      0.00        62

    accuracy                           0.68       192
   macro avg       0.34      0.50      0.40       192
weighted avg       0.46      0.68      0.55       192

Confusion matrix: 
 [[130   0]
 [ 62   0]]
